In [ ]:
import os
import cv2
from ultralytics import YOLO
import xgboost as xgb
import pandas as pd
import time
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

def send_alert_email(frame, timestamp, frame_number):
    """Send email alert with screenshot of suspicious activity"""
    # Email configuration
    sender_email = "****@gmail.com"
    receiver_email = "****@gmail.com"
    password = "**** **** **** ****"  # App-specific password
    
    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"SHOPLIFTING ALERT - Suspicious Activity Detected ({timestamp})"
    
    # Email body
    body = f"""
    SECURITY ALERT: Suspicious activity potentially indicating shoplifting has been detected.
    
    Detection Details:
    - Time: {timestamp}
    - Frame Number: {frame_number}
    
    Please review the attached image and take appropriate action.
    """
    msg.attach(MIMEText(body, 'plain'))
    
    # Attach the frame as image
    _, img_encoded = cv2.imencode('.jpg', frame)
    img_part = MIMEImage(img_encoded.tobytes())
    img_part.add_header('Content-Disposition', 'attachment', filename="suspicious_activity.jpg")
    msg.attach(img_part)
    
    try:
        # Send email
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Alert email sent successfully!")
    except Exception as e:
        print(f"Failed to send email alert: {str(e)}")

def detect_shoplifting_live(camera_index=0, output_folder="output_video"):
    # Load models
    model_yolo = YOLO(r"/Users/*****/Desktop/FYP-shoplifting/FYP-shoplifting/best.pt")
    model = xgb.Booster()
    model.load_model(r"/Users/*****/Desktop/FYP-shoplifting/FYP-shoplifting/model_weights.json")
    
    # Initialize camera
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    if not cap.isOpened():
        for i in range(3):
            cap = cv2.VideoCapture(i, cv2.CAP_AVFOUNDATION)
            if cap.isOpened():
                print(f"Found camera at index {i}")
                break
        else:
            print("Error: Could not open any camera.")
            return
    
    # Camera settings
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 30
    
    # Prepare output
    os.makedirs(output_folder, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(output_folder, f"live_detection_{timestamp}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (1280, 720))
    
    # Alert system
    last_alert_time = 0
    alert_cooldown = 30  # seconds
    frame_count = 0
    
    print("Live detection running... Press 'q' to quit")
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Frame capture error - retrying...")
                time.sleep(0.1)
                continue
            
            frame_count += 1
            results = model_yolo(frame, verbose=False, imgsz=640)
            annotated_frame = results[0].plot(boxes=False)
            
            suspicious_detected = False
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            for r in results:
                boxes = r.boxes.xyxy.cpu().numpy()
                conf = r.boxes.conf.cpu().numpy()
                keypoints = r.keypoints.xyn.cpu().numpy()
                
                for i, (box, cf) in enumerate(zip(boxes, conf)):
                    if cf > 0.75:
                        data = {}
                        for j, (x, y) in enumerate(keypoints[i]):
                            data[f'x{j}'] = x
                            data[f'y{j}'] = y
                        
                        df = pd.DataFrame(data, index=[0])
                        dmatrix = xgb.DMatrix(df)
                        pred = (model.predict(dmatrix) > 0.5).astype(int)[0]
                        
                        x1, y1, x2, y2 = map(int, box[:4])
                        if pred == 0:  # Suspicious
                            suspicious_detected = True
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 7, 58), 2)
                            cv2.putText(annotated_frame, f'Suspicious ({cf:.2f})', 
                                       (x1, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 7, 58), 2)
                            
                            # Send alert if cooldown has passed
                            if (time.time() - last_alert_time) > alert_cooldown:
                                last_alert_time = time.time()
                                print(f"\nALERT: Suspicious activity detected at {current_time}")
                                send_alert_email(frame, current_time, frame_count)
                        else:
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (57, 255, 20), 2)
                            cv2.putText(annotated_frame, f'Normal ({cf:.2f})', 
                                       (x1, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (57, 255, 20), 2)
            
            cv2.imshow('Live Shoplifting Detection', annotated_frame)
            out.write(annotated_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"\nDetection stopped. Recording saved to {output_path}")

# Start detection
detect_shoplifting_live(camera_index=0)

Live detection running... Press 'q' to quit

ALERT: Suspicious activity detected at 2025-04-22 10:30:03
Alert email sent successfully!

ALERT: Suspicious activity detected at 2025-04-22 10:30:33
Alert email sent successfully!

ALERT: Suspicious activity detected at 2025-04-22 10:31:03
Alert email sent successfully!
